In [2]:
from load_docs import load_documents,chunk_documents
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.llms import HuggingFaceHub
import os
import streamlit as st
from langchain.chains import RetrievalQA
from langchain_community.llms import HuggingFaceEndpoint
from transformers import pipeline
from langchain_community.llms import HuggingFacePipeline
from langchain.chains import ConversationalRetrievalChain


/Users/tomthomas/Downloads/RAG Learning/multi_task_rag/rag/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# 1. Load PDF
documents = load_documents("docs/sample.pdf")

In [4]:
# 2. Split into chunks
chunks = chunk_documents(documents)

In [6]:
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-base-en")
vectorstore = Chroma.from_documents(chunks, embeddings)

In [ ]:
hf_pipeline = pipeline(
    "text2text-generation",
    model="google/flan-t5-base",
    device=0  # CPU, or 0 for GPU
)

llm = HuggingFacePipeline(pipeline=hf_pipeline)


OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Meta-Llama-3.1-8B-Instruct.
403 Client Error. (Request ID: Root=1-68b8cf44-3c6f87575137119615e34e9a;3e62d30a-4960-4e81-8bfd-257ca23965ef)

Cannot access gated repo for url https://huggingface.co/meta-llama/Meta-Llama-3.1-8B-Instruct/resolve/main/config.json.
Access to model meta-llama/Llama-3.1-8B-Instruct is restricted and you are not in the authorized list. Visit https://huggingface.co/meta-llama/Llama-3.1-8B-Instruct to ask for access.

In [9]:
# 4. Create QA chain
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
# Point to your deployed Inference Endpoint
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever
)

In [10]:
chat_history = []

query1 = "Who is the Jeff Bezoz?"
result1 = qa_chain({"question": query1, "chat_history": chat_history})
print("Q:", query1)
print("A:", result1["answer"])

# Follow-up vague query
query2 = "What’s his age?"
result2 = qa_chain({"question": query2, "chat_history": chat_history})
print("Q:", query2)
print("A:", result2["answer"])

/var/folders/y6/k28bmfyn2zvc2txjktsxbxhw0000gn/T/ipykernel_20347/2215221939.py:4: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result1 = qa_chain({"question": query1, "chat_history": chat_history})


Q: Who is the Jeff Bezoz?
A: Amazon employees. (Courtesy of Laurel Canan) Jeff Bezos demonstrates an educational toy called Gus Gutz to talk-show host Jay Leno during his appearance on The Tonight Show with Jay Leno at the NBC studios in Burbank on December 29, 1999.
Q: What’s his age?
A: 20-year-old
